In [1]:
import cv2
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import random
import numpy as np
from PIL import Image


In [ ]:

# Step 1: Load the .mov video file
video_path = '../Screen Recording 2024-03-07 at 10.25.19 AM.mov'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
random_frame_number = random.randint(0, frame_count - 1)

# Step 2: Sample a random frame from the video

frames = []

for f in range(frame_count//50):
    if f%10==0:
        print(f,"/",frame_count)
    #cap.set(cv2.CAP_PROP_POS_FRAMES, f)
    success, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    original_height, original_width = frame.shape[:2]

    # Calculate new dimensions (half)
    new_width = original_width // 2
    new_height = original_height // 2
    
    # Resize the frame to half of the original dimensions
    resized_frame = cv2.resize(frame_rgb, (new_width, new_height), interpolation=cv2.INTER_AREA)

    if success:
        frames.append(resized_frame)
    

In [ ]:
video_path = '../Screen Recording 2024-03-07 at 10.25.19 AM.mov'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Step 2: Sample a random frame from the video

frames = []
frame_number=0
# Loop through the frames, skipping every 30 frames
while frame_number < total_frames:
    print(frame_number, "/", total_frames)
    # Set the video capture object to the current frame number
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Read the frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if ret:
        # Convert the frame to RGB color space
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get the original dimensions of the frame
        original_height, original_width = frame.shape[:2]

        # Calculate new dimensions (half)
        new_width = original_width // 2
        new_height = original_height // 2

        # Resize the frame to half of the original dimensions
        resized_frame = cv2.resize(frame_rgb, (new_width, new_height), interpolation=cv2.INTER_AREA)

        # Append the resized frame to the list
        frames.append(resized_frame)
    else:
        break

    # Update the frame number to skip every 30 frames
    frame_number += 30

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
frames = np.array(frames)
np.save('frames.npy', frames)


In [2]:
frames=np.load('frames.npy')

In [3]:
import argparse
from fastsam import FastSAM, FastSAMPrompt 
import ast
import torch
from PIL import Image
from utils.tools import convert_box_xywh_to_xyxy


In [4]:
from PIL import Image
from scipy.spatial import cKDTree


def sm(im):
    Image.fromarray(im).show()

In [5]:

class args_init:
    def __init__(self):
        self.model_path = "./FastSAM.pt"
        self.img_path = "./images/imageForSegmentation.png"
        self.imgsz = 1024
        self.iou = 0.9
        self.text_prompt = None

        self.conf = 0.4
        self.output = "./output/"
        self.point_prompt="[[0,0]]"

        self.randomcolor = True


        self.point_label = "[0]"
        self.box_prompt = "[[0,0,0,0]]"
        self.better_quality = False
        self.device = torch.device(
            "cuda"
            #if torch.cuda.is_available()
            #else "mps"
            #if torch.backends.mps.is_available()
            #else "cpu"
        )
        self.retina = True
        self.withContours = False

args = args_init()

In [6]:

model = FastSAM(args.model_path)
args.point_prompt = ast.literal_eval(args.point_prompt)
args.box_prompt = convert_box_xywh_to_xyxy(ast.literal_eval(args.box_prompt))
args.point_label = ast.literal_eval(args.point_label)

bboxes = None
points = None
point_label = None



In [7]:
# load model

def run_SAM(frame):
    everything_results = model(
    frame,
    device=args.device,
    retina_masks=args.retina,
    imgsz=args.imgsz,
    conf=args.conf,
    iou=args.iou    
    )

    prompt_process = FastSAMPrompt(frame, everything_results, device=args.device)
    ann = prompt_process.everything_prompt()
    return ann




In [8]:
import numpy as np
import cv2



#def render_com(mask):
#    cX, cY = get_center_of_mass(mask)
#    image_colored = cv2.cvtColor(mask*255.0, cv2.COLOR_GRAY2BGR)

#    image_colored = cv2.circle(image_colored, (cX, cY), 10, (0, 0, 255), -1)  # Red circle

#    sm(cv2.convertScaleAbs(image_colored))
        


In [ ]:
run_SAM(Image.fromarray(frames[200]).convert("RGB"))

In [9]:
subset = 100


In [10]:
moments = []
annons = []
for (idx,f) in enumerate(range(len(frames))):
    ann = run_SAM(Image.fromarray(frames[f]).convert("RGB"))
    #moments.append(get_moments(ann))
    #print(ann)
    annons.append([a.cpu().detach().numpy() for a in ann])
    print(idx,"/",len(frames))
    if idx>subset:
        break




0: 576x1024 99 objects, 231.4ms
Speed: 2.2ms preprocess, 231.4ms inference, 201.2ms postprocess per image at shape (1, 3, 1024, 1024)



0 / 288


0: 576x1024 98 objects, 190.2ms
Speed: 0.0ms preprocess, 190.2ms inference, 38.5ms postprocess per image at shape (1, 3, 1024, 1024)



1 / 288


0: 576x1024 98 objects, 193.7ms
Speed: 0.0ms preprocess, 193.7ms inference, 27.0ms postprocess per image at shape (1, 3, 1024, 1024)



...

99 / 288


0: 576x1024 92 objects, 253.1ms
Speed: 8.6ms preprocess, 253.1ms inference, 29.7ms postprocess per image at shape (1, 3, 1024, 1024)



100 / 288


0: 576x1024 91 objects, 240.0ms
Speed: 7.0ms preprocess, 240.0ms inference, 38.0ms postprocess per image at shape (1, 3, 1024, 1024)


101 / 288


In [64]:
def get_center_of_mass(mask):
    mask = mask.astype('uint8')
    M = cv2.moments(mask)
    if M["m00"] == 0:
        return (0,0)
    
    moments=np.array([M[m] for m in ["m10", "m01"]])#, "m20", "m11", "m02", "m30", "m21", "m12", "m03"]])
    return moments/M["m00"]
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    return (cX, cY)


def get_moments(ann):
    annotation_locs = []
    for a in range(len(ann)):
        annotation_locs.append(get_center_of_mass(ann[a]))
    return np.array(annotation_locs)

moments = []
for (f,ann) in enumerate(annons):
    #moments.append([])
    annotation_locs = []
    for a in range(len(ann)):

        avg_color = frames[f][ann[a]==1].sum(axis=0)/ann[a].sum()
        annotation_locs.append(list(get_center_of_mass(ann[a]))+list(avg_color))
        
    #print(annotation_locs)
    
    moments.append(np.array(annotation_locs))

In [46]:
len(annons)

102

In [25]:
moments

[array([[     1287.7,      144.87,  1.6604e+06,  1.8703e+05,       26317,  2.1436e+09,  2.4184e+08,  3.4088e+07,  5.2975e+06],
        [     176.91,      711.04,       38531,  1.2618e+05,  5.0693e+05,  9.4927e+06,   2.759e+07,  9.0212e+07,  3.6236e+08],
        [     859.65,      460.08,  7.4481e+05,  3.9579e+05,  2.1197e+05,  6.5034e+08,   3.432e+08,  1.8248e+08,  9.7797e+07],
        [     1181.9,      51.472,  1.3975e+06,       60681,      3442.2,  1.6532e+09,  7.1569e+07,  4.0523e+06,  2.5995e+05],
        [     471.22,      451.03,  2.2371e+05,  2.1254e+05,  2.0377e+05,  1.0697e+08,   1.009e+08,  9.6025e+07,  9.2218e+07],
        [     542.28,      747.32,  2.9503e+05,  4.0521e+05,   5.587e+05,  1.6103e+08,  2.2043e+08,   3.029e+08,  4.1784e+08],
        [     346.65,      191.93,  1.2079e+05,       66594,       39098,    4.23e+07,  2.3223e+07,  1.3577e+07,  8.3787e+06],
        [     372.52,         577,  1.4002e+05,  2.1304e+05,  3.3642e+05,   5.309e+07,  7.9373e+07,  1.2313e+08

In [65]:
moments

[array([[     1287.7,      144.87,      121.02,       43.05,      39.334],
        [     176.91,      711.04,      121.52,      89.755,      25.066],
        [     859.65,      460.08,      149.34,      130.84,      89.429],
        [     1181.9,      51.472,      77.937,      74.474,      66.665],
        [     471.22,      451.03,      114.19,      168.06,      133.92],
        [     542.28,      747.32,      87.612,      77.458,      47.839],
        [     346.65,      191.93,      128.31,      130.33,      130.66],
        [     372.52,         577,       40.77,      37.379,      28.817],
        [     247.88,      240.77,       130.4,      135.77,      136.93],
        [     970.03,       231.2,      35.398,      28.898,       23.55],
        [     909.01,       580.2,      158.64,      149.55,      125.41],
        [     662.06,      619.57,      114.43,      83.822,      25.878],
        [     1083.8,      51.976,      132.57,      112.71,      81.411],
        [     998.69,    

In [ ]:
from scipy.spatial import cKDTree


In [ ]:
frames_annotate = frames.copy()

for g in groups:
    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    for a in groups[g]:
        idx = a[0]
        cX, cY = a[1][:2]
        frames_annotate[idx] = cv2.circle(frames_annotate[idx], (int(cX), int(cY)), 10, color, -1)

In [138]:
trackers = moments[0]
last_seen = [0 for t in trackers]
colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for t in trackers]

for (idx,m) in enumerate(moments[1:]):
    
    tree = cKDTree(trackers)
    dists, neighbors = tree.query(m)
    
    canidate_updates = {}
    for (i,n) in enumerate(neighbors):
        if dists[i]<100 and last_seen[n]<12:
            if n not in canidate_updates:
                canidate_updates[n] = [i]
            else:
                canidate_updates[n].append(i)
        else:
            trackers=np.append(trackers,m[i].reshape(1,5),axis=0)
            last_seen.append(0)
            colors.append((random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))

        last_seen[n] += 1
    
    for c in canidate_updates:
        last_seen[c] = 0
        
        min_idx = canidate_updates[c][np.argmin([dists[can] for can in canidate_updates[c]])]
        
        trackers[c] = m[min_idx]
        cX, cY = m[min_idx][:2]
        color = (m[min_idx][2],m[min_idx][3],m[min_idx][4])#colors[c]
        frames_annotate[idx] = cv2.circle(frames_annotate[idx], (int(cX), int(cY)), 10, color, -1)
        frames_annotate[idx] = cv2.circle(frames_annotate[idx], (int(cX), int(cY)), 10, colors[c], 4)



In [142]:
num_frames, height, width, channels = frames_annotate.shape

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec as needed
fps = 5.0  # Frames per second
video_writer = cv2.VideoWriter('output_video.mp4', fourcc, fps, (width, height))

# Loop through the frames and write them to the video
for frame in frames_annotate[:100]:
    # Convert the frame from NumPy array to OpenCV format
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # Write the frame to the video
    video_writer.write(frame)

# Release the video writer object
video_writer.release()

In [140]:
from scipy.spatial import cKDTree

def get_diff(ann1, ann2):
    tree = cKDTree(ann1)
    ret = tree.query(ann2)
    
    return ret


In [43]:
groups = {"random_id+"+str(i):[1,(0,moments[0][i])] for i in range(len(moments[0]))}

gmap = {i:"random_id+"+str(i) for i in range(len(moments[0]))}

#Have blobs that remain for as many frames as they were there. If they haven't been there for that long, remove them.

current_blobs = {i:0 for i in range(len(moments[0]))}

for (idx,f) in enumerate(range(len(moments)-1)):
    if len(moments[f+1])>0:
        #pass in all blobs that haven't timed out
        ret = get_diff(moments[f], moments[f+1])
    else:
        continue
    if ret==None:
        continue
    dists, neighbors = ret
    #print(dists)
    gmap2 = {}
    for i in range(len(neighbors)):
        #id -> past id
        if neighbors[i] in gmap and dists[i]<1000:
            gmap2[i] = gmap[neighbors[i]]
            current_blobs[]
            groups[gmap2[i]].append((idx,moments[f+1][i]))
        else:
            gmap2[i] = "random_id_n+"+str(idx)+"-"+str(i)
            groups[gmap2[i]] = [(idx,moments[f+1][i])]
    gmap=gmap2


In [30]:
#TODO: apply to more frames
#TODO: use all moment points to be k-dimensional not just center of mass

array([     176.91,      711.04,       38531,  1.2618e+05,  5.0693e+05,  9.4927e+06,   2.759e+07,  9.0212e+07,  3.6236e+08])

In [31]:
#for (idx,f) in enumerate(range(8)):
    
    #mask=annons[idx][groups["random_id+10"][idx]]
    #print(mask.shape)
    #sm(mask.to("cpu").numpy()*255)
    #frames[idx]
    #

In [37]:
frames_annotate = frames.copy()



for g in groups:
    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    for a in groups[g]:
        idx = a[0]
        cX, cY = a[1][:2]
        frames_annotate[idx] = cv2.circle(frames_annotate[idx], (int(cX), int(cY)), 10, color, -1)